In [22]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, GaussianDropout
from tensorflow.keras import regularizers
from tensorflow.keras.layers import Dense, Dropout, LeakyReLU, PReLU, ELU
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from keras.layers import Activation



In [23]:
df = pd.read_csv('https://raw.githubusercontent.com/jbrownlee/Datasets/master/monthly-sunspots.csv')
df.head(10)

,Month,Sunspots
0,1749-01,58.0
1,1749-02,62.6
2,1749-03,70.0
3,1749-04,55.7
4,1749-05,85.0
5,1749-06,83.5
6,1749-07,94.8
7,1749-08,66.3
8,1749-09,75.9
9,1749-10,75.5


In [24]:
# Convert the "year-month" column to a datetime format
df['Month'] = pd.to_datetime(df['Month'])

# Extract year and month components into separate columns
df['year'] = df['Month'].dt.year
df['month'] = df['Month'].dt.month

# Drop the original "year_month" column if needed
df.drop(columns=['Month'], inplace=True)

# Print the resulting DataFrame
print(df)

      Sunspots  year  month
0         58.0  1749      1
1         62.6  1749      2
2         70.0  1749      3
3         55.7  1749      4
4         85.0  1749      5
...        ...   ...    ...
2815      71.8  1983      8
2816      50.3  1983      9
2817      55.8  1983     10
2818      33.3  1983     11
2819      33.4  1983     12

[2820 rows x 3 columns]


In [25]:
scaler = StandardScaler()
X = df.drop(["Sunspots"], axis=1).values
X = scaler.fit_transform(X)

y = df[['Sunspots']].values
y = scaler.fit_transform(y)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model3 = Sequential()


model3.add(Dense(256, activation='tanh', kernel_regularizer=regularizers.l2(0.01)))
model3.add(Dropout(0.2))

model3.add(Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model3.add(Dropout(0.2))

model3.add(Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model3.add(Dropout(0.2))

model3.add(Dense(32, activation='tanh', kernel_regularizer=regularizers.l2(0.01)))
model3.add(Dropout(0.2))
model3.add(Dense(16, activation='tanh', kernel_regularizer=regularizers.l2(0.01)))
model3.add(Dropout(0.2))

model3.add(Dense(1, activation='linear'))

In [26]:
model3.compile(optimizer='adam', loss='mean_squared_error')
history3 = model3.fit(X_train, y_train, epochs=100, batch_size=80, validation_split=0.25, verbose=2)


print("Perdida de entrenamiento del modelo 1:" + str({history3.history['loss'][-1]}) + "\n")
loss3 = model3.evaluate(X_test, y_test)
print("Perdida de prueba del modelo 1:" + str(loss3))

Epoch 1/100


22/22 - 2s - loss: 3.9927 - val_loss: 3.5328 - 2s/epoch - 71ms/step
Epoch 2/100
22/22 - 0s - loss: 3.4346 - val_loss: 3.0198 - 130ms/epoch - 6ms/step
Epoch 3/100
22/22 - 0s - loss: 2.9616 - val_loss: 2.6011 - 107ms/epoch - 5ms/step
Epoch 4/100
22/22 - 0s - loss: 2.5756 - val_loss: 2.2631 - 201ms/epoch - 9ms/step
Epoch 5/100
22/22 - 0s - loss: 2.2648 - val_loss: 1.9833 - 106ms/epoch - 5ms/step
Epoch 6/100
22/22 - 0s - loss: 2.0058 - val_loss: 1.7569 - 127ms/epoch - 6ms/step
Epoch 7/100
22/22 - 0s - loss: 1.8099 - val_loss: 1.5776 - 119ms/epoch - 5ms/step
Epoch 8/100
22/22 - 0s - loss: 1.6519 - val_loss: 1.4360 - 98ms/epoch - 4ms/step
Epoch 9/100
22/22 - 0s - loss: 1.5242 - val_loss: 1.3405 - 100ms/epoch - 5ms/step
Epoch 10/100
22/22 - 0s - loss: 1.4251 - val_loss: 1.2381 - 102ms/epoch - 5ms/step
Epoch 11/100
22/22 - 0s - loss: 1.3493 - val_loss: 1.1699 - 147ms/epoch - 7ms/step
Epoch 12/100
22/22 - 0s - loss: 1.2748 - val_loss: 1.1162 - 171ms/epoch - 8ms/step
Epoch 13/100
22/22 - 0s - lo

In [29]:
y_pred = model3.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Print the evaluation metrics
print("Mean Squared Error (MSE):", mse)
print("Mean Absolute Error (MAE):", mae)
print("R-squared (R²) Score:", r2)

18/18 [==============================] - 0s 2ms/step
Mean Squared Error (MSE): 0.926575789711941
Mean Absolute Error (MAE): 0.7805591801311786
R-squared (R²) Score: 0.05068560930636545


In [30]:
y_pred = model3.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Print the evaluation metrics
print("Mean Squared Error (MSE):", mse)
print("Mean Absolute Error (MAE):", mae)
print("R-squared (R²) Score:", r2)

18/18 [==============================] - 0s 1ms/step
Mean Squared Error (MSE): 0.926575789711941
Mean Absolute Error (MAE): 0.7805591801311786
R-squared (R²) Score: 0.05068560930636545
